In [28]:
!pip install kmodes
!pip install gower


In [2]:
from ucimlrepo import fetch_ucirepo 
  
# fetch dataset 
adult = fetch_ucirepo(id=2) 
  
# data (as pandas dataframes) 
X = adult.data.features 
y = adult.data.targets 
  
df = X


In [5]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler, MaxAbsScaler

df: pd.DataFrame = X

print(df.shape)
df = df[~df.eq("?").any(axis=1)]
df = df.dropna().reset_index(drop=True)

print(df.shape)

df.to_csv("adults.csv", index=False)

# Identify categorical and numerical columns
categorical_cols = df.select_dtypes(include=['object']).columns
numerical_cols = df.select_dtypes(include=['int64', 'float64']).columns


# df = pd.get_dummies(df, columns=categorical_cols)

# cat_cols = df.select_dtypes(include=['object', 'category']).columns
categorical_indexes = [df.columns.get_loc(col) for col in categorical_cols]

categorical_indexes

for col in categorical_cols:
    df[col], _ = pd.factorize(df[col])
    
# df[numerical_cols]  = MinMaxScaler().fit_transform(df[numerical_cols])
    
df

(48842, 14)
(45222, 14)


,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country
0,39,0,77516,0,13,0,0,0,0,0,2174,0,40,0
1,50,1,83311,0,13,1,1,1,0,0,0,0,13,0
2,38,2,215646,1,9,2,2,0,0,0,0,0,40,0
3,53,2,234721,2,7,1,2,1,1,0,0,0,40,0
4,28,2,338409,0,13,1,3,2,1,1,0,0,40,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45217,33,2,245211,0,13,0,3,3,0,0,0,0,40,0
45218,39,2,215419,0,13,2,3,0,0,1,0,0,36,0
45219,38,2,374983,0,13,1,3,1,0,0,0,0,50,0
45220,44,2,83891,0,13,2,0,3,2,0,5455,0,40,0


In [ ]:
import gower
from sklearn.cluster import DBSCAN
from sklearn.metrics import silhouette_score

new_df: pd.DataFrame = X

new_df = new_df[~new_df.eq("?").any(axis=1)]
new_df = new_df.dropna().reset_index(drop=True)
mini_df = new_df.sample(10000)

gower_dist = gower.gower_matrix(mini_df)
full_results = []

epsilon_values = np.arange(0.05, 1.05, 0.05)
min_samples_values = range(2, 101, 2)

max_score = -1
# Run DBSCAN on the Gower distance matrix

for epsilon in epsilon_values:
    for min_sample in min_samples_values:
        dbscan = DBSCAN(metric='precomputed', eps=epsilon, min_samples=min_sample)
        labels = dbscan.fit_predict(gower_dist)
        number_of_labels = len(set(labels))
        if ({-1, 0} == set(labels)):
            print(f"{epsilon=}, {min_sample=} classification is bad")
            score_with_bad = silhouette_score(gower_dist, labels, metric='precomputed')
            print(f"{epsilon=}, {min_sample=}, {score_with_bad=}")
            continue
        if number_of_labels == 1:
            print(f"{epsilon=}, {min_sample=} classification is very bad")
            continue

        mask = labels != -1
        try:
        # Compute silhouette score only on non-noise points
            score_without_bad = silhouette_score(gower_dist[mask][:, mask], labels[mask], metric='precomputed')
            score_with_bad = silhouette_score(gower_dist, labels, metric='precomputed')
        except Exception as e:
            import ipdb
            ipdb.set_trace()
        if score_with_bad >= max_score:
            max_score = score_with_bad
            print(f"{epsilon=}, {min_sample=}, {number_of_labels=}, {score_with_bad=}, {score_without_bad}")
            
        full_results.append([epsilon, min_sample, number_of_labels, score_with_bad, score_without_bad])
        
import pickle


with open("gower_matrix_dbscan.pkl", "wb") as f:
    pickle.dump(full_results, f)




epsilon=0.05, min_sample=2, number_of_labels=1037, score_with_bad=0.32259753, 0.7163484692573547
epsilon=0.1, min_sample=8 classification is bad
epsilon=0.1, min_sample=8, score_with_bad=0.20074089
epsilon=0.1, min_sample=14 classification is bad
epsilon=0.1, min_sample=14, score_with_bad=0.19211902
epsilon=0.1, min_sample=18 classification is bad
epsilon=0.1, min_sample=18, score_with_bad=0.18652338
epsilon=0.1, min_sample=20 classification is bad
epsilon=0.1, min_sample=20, score_with_bad=0.18560497
epsilon=0.1, min_sample=22 classification is bad
epsilon=0.1, min_sample=22, score_with_bad=0.18380935
epsilon=0.1, min_sample=24 classification is bad
epsilon=0.1, min_sample=24, score_with_bad=0.1824984
epsilon=0.1, min_sample=26 classification is bad
epsilon=0.1, min_sample=26, score_with_bad=0.1810911
epsilon=0.1, min_sample=28 classification is bad
epsilon=0.1, min_sample=28, score_with_bad=0.17873658
epsilon=0.1, min_sample=30 classification is bad
epsilon=0.1, min_sample=30, score_

TypeError: Object of type float32 is not JSON serializable

In [71]:
mask = labels != -1

# Compute silhouette score only on non-noise points
score = silhouette_score(gower_dist[mask][:, mask], labels[mask], metric='precomputed')
# score = silhouette_score(gower_dist, labels, metric='precomputed')

print("Silhouette Score:", score)

Silhouette Score: -0.016717758


In [11]:
from kmodes.kprototypes import KPrototypes
from sklearn.metrics import silhouette_score
import numpy as np

# Define the range of potential clusters and gamma values
clusters_range = range(2,15)
gamma_range = np.linspace(0.1, 1, 10) # gamma values between 0.1 and 1

# Placeholder variables
best_score = -1
best_clusters = None
best_labels = None
best_gamma = None

all_scores = []

mini_df = df.sample(10000)

for n_clusters in clusters_range:
    # for gamma in gamma_range:
    kproto = KPrototypes(n_clusters=n_clusters, init='Cao')
    clusters = kproto.fit_predict(mini_df, categorical=categorical_indexes)
    score = silhouette_score(mini_df, clusters)
    all_scores.append([n_clusters, score, kproto.cost_])

    # Check if this configuration beats the best score
    print(f"{score=}, {n_clusters=}, {kproto.cost_=}")
    if score > best_score:
        best_labels = clusters
        best_score = score
        best_clusters = n_clusters
        # best_gamma = gamma

print(f"Best score: {best_score}")
print(f"Optimal number of clusters: {best_clusters}")
print(f"Optimal gamma value: {best_gamma}")
print(f"{all_scores}")

score=0.5839696420851198, n_clusters=2, kproto.cost_=47600340443889.32
score=0.5470239186384337, n_clusters=3, kproto.cost_=26250635086135.605
score=0.5503478123708605, n_clusters=4, kproto.cost_=17651557379011.86
score=0.5309211677627476, n_clusters=5, kproto.cost_=12930092690963.555
score=0.5350708561075785, n_clusters=6, kproto.cost_=9669832064624.227
score=0.5425061502647003, n_clusters=7, kproto.cost_=7034531985936.423
score=0.5285074019016304, n_clusters=8, kproto.cost_=5450677488713.95
score=0.5277898745868209, n_clusters=9, kproto.cost_=4397565147883.036
score=0.5266726647543968, n_clusters=10, kproto.cost_=3729302490553.4453


KeyboardInterrupt: 

In [ ]:
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import seaborn as sns

tsne = TSNE(n_components=2, random_state=53, perplexity=50)
tsne_on_df = tsne.fit_transform(mini_df)

def plot_tsne(X_tsne, title):
    plt.figure(figsize=(8,6))
    sns.scatterplot(x=X_tsne[:,0], y=X_tsne[:,1], hue=y_sample, palette='viridis', s=50)
    plt.title(title)
    plt.xlabel('t-SNE Component 1')
    plt.ylabel('t-SNE Component 2')
    plt.legend(title='Income')
    plt.show()

# Visualizations
plot_tsne(X_tsne_pca, 't-SNE on PCA-Reduced Data')